In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Thanks to @pratikkgandhi for baseline HistGradient classifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold, StratifiedShuffleSplit
import matplotlib.pyplot as plt
from sklearn.preprocessing import QuantileTransformer , RobustScaler , MinMaxScaler , StandardScaler,PowerTransformer
from sklearn.metrics import roc_auc_score , roc_curve
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier 

import optuna

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/test.csv")
submission = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv")

In [ ]:
X = train.drop(['id','claim'],axis = 1)
y = train['claim']

test_data = test.drop('id',axis = 1).copy()

In [ ]:
# Credits to @hiro5299834
X['n_missing'] = X.isna().sum(axis=1)
test_data['n_missing'] = test_data.isna().sum(axis=1)

X['min'] = X.min(axis=1)
test_data['min'] = test_data.min(axis=1)

X['max'] = X.max(axis=1)
test_data['max'] = test_data.max(axis=1)

X['std'] = X.std(axis=1)
test_data['std'] = test_data.std(axis=1)



X['avg'] = X.mean(axis = 1)
test_data['avg'] = test_data.mean(axis = 1)




In [ ]:
imputer = SimpleImputer()

X_imputed =  imputer.fit_transform(X)
test_imputed =  imputer.transform(test_data)

In [ ]:
sc = RobustScaler()
sc.fit(X_imputed)
X_scaled = pd.DataFrame(sc.transform(X_imputed), columns=X.columns)
X_test_scaled = pd.DataFrame(sc.transform(test_imputed), columns=test_data.columns)

In [ ]:
# %%time
# def objective(trial):
#     X_train,X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, shuffle=True)
    
#     # Parameters
#     params = {
        
#         "loss":trial.suggest_categorical("loss",["binary_crossentropy"]),
#         "learning_rate": trial.suggest_float("learning_rate", 0.01,0.3),
#         "max_iter" : trial.suggest_int("max_iter", 50,400),
#         "max_leaf_nodes":trial.suggest_int("max_leaf_nodes",10,60),
#         "min_samples_leaf" : trial.suggest_int("min_samples_leaf", 2,40),
#         "l2_regularization": trial.suggest_float("l2_regularization", 0.0,0.5),
#         "max_bins":trial.suggest_int("max_bins",2,255)
#         }
#     model = HistGradientBoostingClassifier(  **params) 
        
#     model.fit(X_train, y_train)

#     # # Predict
#     preds = model.predict_proba(X_test)[:,1]
#     # # eval
#     roc_score = roc_auc_score(y_test,preds)
 
#     return roc_score

In [ ]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=100)

In [ ]:
# print("Number of finished trials: {}".format(len(study.trials)))
# print("Best trial:")
# trial = study.best_trial
# print("  Value: {}".format(trial.value))
# print("  Params: ")
# for key, value in trial.params.items():
#   print("    {}: {}".format(key, value))

In [ ]:
# hg_params = {
#     'loss': 'binary_crossentropy',
#     'learning_rate': 0.08157930234173807,
#     'max_iter': 359,
#     'max_leaf_nodes': 23,
#     'min_samples_leaf': 17,
#     'l2_regularization': 0.19491384748432528,
#     'max_bins': 244,
#    }
# test_preds = 0
# train_oof = np.zeros((X_scaled.shape[0],1))
# train_oof = pd.DataFrame(train_oof, columns=['preds'])
# kf = KFold(n_splits=5)
# for train_index, test_index in kf.split(X_scaled):
#     clf = HistGradientBoostingClassifier(**hg_params)
#     trn_X, trn_y = X_scaled.loc[train_index], y.loc[train_index]
#     val_X, val_y = X_scaled.loc[test_index], y.loc[test_index]
#     clf.fit(trn_X,trn_y)
#     oof_proba = clf.predict_proba(val_X)[:,1]
#     train_oof.loc[test_index,'preds'] = oof_proba
#     tmp = clf.predict_proba(X_test_scaled)
#     test_preds += tmp/5
    
# print("The score for OOF is:")
# roc_auc_score(train['claim'], train_oof['preds'])

In [ ]:
hg1 = {
    'l2_regularization': 3.5676998099580946e-05,
    'early_stopping': 'False',
    'learning_rate': 0.036266802532849,
    'max_iter': 1000,
    'max_depth': 20,
    'max_bins': 234,
    'min_samples_leaf': 10000,
    'max_leaf_nodes': 48
}

test_preds = 0
train_oof = np.zeros((X_scaled.shape[0],1))
train_oof = pd.DataFrame(train_oof, columns=['preds'])
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X_scaled):
    clf = HistGradientBoostingClassifier(**hg1)
    trn_X, trn_y = X_scaled.loc[train_index], y.loc[train_index]
    val_X, val_y = X_scaled.loc[test_index], y.loc[test_index]
    clf.fit(trn_X,trn_y)
    oof_proba = clf.predict_proba(val_X)[:,1]
    train_oof.loc[test_index,'preds'] = oof_proba
    tmp = clf.predict_proba(X_test_scaled)
    test_preds += tmp/5
    
print("The score for OOF is:")
roc_auc_score(train['claim'], train_oof['preds'])

In [ ]:
submission['claim'] = test_preds[:,1]
submission.to_csv('histogram_new.csv', index=False)

In [ ]:
submission.head()